In [ ]:
%load_ext autoreload
%autoreload 2
from lib import load, timing

YEAR = 2024
DAY = 17
TESTDATA = [
    None,
    'Register A: 729\nRegister B: 0\nRegister C: 0\n\nProgram: 0,1,5,4,3,0\n'
][0]
TEST = TESTDATA is None

In [ ]:
import re

# Parser:
re_register = re.compile(r'Register A: (\d+)')
re_program = re.compile(r'Program: (.+)')
@timing
def prepare_data():
    data = load(YEAR, DAY, split_lines=True, test=TESTDATA if TEST else None)

    result = re_register.search(data['split'][0])
    register = int(result.group(1))
        
    result = re_program.search(data['split'][4])
    program = [int(t.strip()) for t in result.group(1).split(',')]

    return register, program

In [ ]:
# Level 1
def op_combo(operand, state):
    num = operand & 3
    if num == operand:
        return num
    else:
        return state[num]
        
def op_literal(operand, state):
    return operand

def adv(operand, state):
    state[0] = state[0] // (2 ** operand)
    
def bxl(operand, state):
    state[1] = state[1] ^ operand

def bst(operand, state):
    state[1] = operand & 7

def jnz(operand, state):
    if state[0] != 0:
        state[3] = operand

def bxc(operand, state):
    state[1] = state[1] ^ state[2]

def out(operand, state):
    state[4].append(operand & 7)

def bdv(operand, state):
    state[1] = state[0] // (2 ** operand)

def cdv(operand, state):
    state[2] = state[0] // (2 ** operand)

def execute(state, program):
    commands = {
        0: {'action': adv, 'operand': op_combo},
        1: {'action': bxl, 'operand': op_literal},
        2: {'action': bst, 'operand': op_combo},
        3: {'action': jnz, 'operand': op_literal},
        4: {'action': bxc, 'operand': op_literal},
        5: {'action': out, 'operand': op_combo},
        6: {'action': bdv, 'operand': op_combo},
        7: {'action': cdv, 'operand': op_combo}
    }
    cmd = commands[program[state[3]]]
    state[3] += 1
    opr = program[state[3]]
    state[3] += 1
    state = cmd['action'](cmd['operand'](opr, state), state)
    
def run(a, program):
    state = [a, 0, 0, 0, []]
    while state[3] < len(program):
        execute(state, program)
    return state[4]


@timing
def level1(register, program):
    output = run(register, program)
    return ','.join([str(i) for i in output])

print(level1(*prepare_data()))

# Theorie:

## Program:
24, 12, 75, 47, 13, 55, 03, 30
   1.  2, 4 > bst A > B = A & 7
   2.  1, 2 > bxl 2 > B = B ^ 2
   3.  7, 5 > cdv B > C = A >> B
   4.  4, 7 > bxc 7 > B = B ^ C
   5.  1, 3 > bxl 3 > B = B ^ 3
   6.  5, 5 > out B > print: B & 7
   7.  0, 3 > adv 3 > A = A >> 3
   8.  3, 0 > jnz 0 > if (A != 0) -> 0
   
## First iteration
   1-2:  B = (A & 7) ^ 2
   3:    C = A >> B
   4.  B = B ^ C
   5.  B = B ^ 3
   6.  print: B & 7
   7.  A = A >> 3
   8.  if (A != 0) -> 0
   


## Conclusion:

A ist built from 16 3-bit groups. After each run through the program, the lowest group is removed. Therefore, only the last group affects the last printout. So, the number groups can be guessed back to front.

In [ ]:
# Level 2
@timing
def level2(register, program):
    a_fixed = [0]
    for start in range(len(program) - 1, -1, -1):
        next_a = []
        for a_now in a_fixed:
            # build A and execute
            for a in range(a_now * 8, (a_now + 1) * 8):
                output = run(a, program)
                if program[start:] == output:
                    next_a.append(a)
        a_fixed = next_a
    
    return min(a_fixed)

print(level2(*prepare_data()))